In [3]:
import requests
import json
import os

from dotenv import load_dotenv
load_dotenv()
date = '2022-08-11'


In [4]:

# get all the ids of a match for a url 
url = "https://v1.baseball.api-sports.io/games"

payload={'league':1, 'season':2022,'date':date}

headers = {
    'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY"),
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

game_response = json.loads(requests.request("GET", url, headers=headers, params=payload).text)


In [18]:
game_response['response'][0]['teams']['home']['name'].split()[1]

'Astros'

In [19]:
score_match_dict = {}
for match in game_response['response']:
    # get the fixture id
    fixture = match['id']
    
     # get the team names
    home_team = match['teams']['home']['name'].split()[1]
    away_team = match['teams']['away']['name'].split()[1]
    # get the score
    home_score = match['scores']['home']['hits']
    away_score = match['scores']['away']['hits']

    # who won?
    if home_score > away_score:
        winner = 'Home'
    elif home_score < away_score:
        winner = 'Away'
    else:
        winner = 'Draw'

    score_match_dict[fixture]={'home_team':home_team, 'away_team':away_team, 'home_score':home_score, 'away_score':away_score, 'winner':winner}


In [21]:
url = "https://v1.baseball.api-sports.io/odds"

payload={'league':1, 'season':2022}

headers = {
    'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY"),
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

odds_response = json.loads(requests.request("GET", url, headers=headers, params=payload).text)


In [22]:
odds_match_dict={}
for match in odds_response['response']:
    if match['game']['id'] in score_match_dict.keys():
        assert(match['bookmakers'][0]['name']=='NordicBet')
        assert(match['bookmakers'][0]['bets'][0]['name']=='Match Winner')

        odd_dict = {}
        for value in match['bookmakers'][0]['bets'][0]['values']:
            odd_dict[value['odd']] = value['value']

        pred_win = odd_dict[min(odd_dict.keys())]
        odds_match_dict[match['game']['id']] = pred_win
    else:
        pass



In [37]:
# get lists of correct and incorrect predictions
correct_ids = []
upset_ids = []

for match_id in odds_match_dict.keys():
    if odds_match_dict[match_id]!=score_match_dict[match_id]['winner']:
       upset_ids.append(match_id) 
    else:
        correct_ids.append(match_id)

def create_tweet(correct_ids, upset_ids, score_match_dict, date='2022-08-11'):
    tweet_text = str('#MLB Scores {date}\n\n').format(date=date)
    tweet_text=tweet_text+str("UPSETS: \n")
    for id in upset_ids:
        score_match_dict[id]
        tweet_text=tweet_text+str("{home_team}: {home_score} - {away_team}: {away_score} \n").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score'])

    tweet_text=tweet_text+str("\n EXPECTED: \n")
    for id in correct_ids:
        score_match_dict[id]
        tweet_text=tweet_text+(str("{home_team}: {home_score} - {away_team}: {away_score} \n").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score']))

    if len(tweet_text)>=280:
        tweet_text_list = tweet_text.split('EXPECTED')
        tweet_text_list[1]='EXPECTED'+tweet_text_list[1]
    else:
        tweet_text_list=[tweet_text]

    return tweet_text_list

tweet_text_list = create_tweet(correct_ids, upset_ids, score_match_dict, date='2022-08-11')

In [44]:
for tweet_text in (tweet_text_list):
    print(tweet_text)

#MLB Scores 2022-08-11

UPSETS: 
Astros: 8 - Rangers: 12 
City: 15 - White: 7 
Diamondbacks: 7 - Pirates: 11 
Phillies: 7 - Marlins: 7 
Red: 8 - Orioles: 8 
Reds: 7 - Cubs: 10 

 
EXPECTED: 
Rockies: 13 - Cardinals: 18 
Angeles: 9 - Twins: 8 
Tigers: 7 - Guardians: 8 
City: 8 - White: 14 
Astros: 10 - Rangers: 8 
Diamondbacks: 9 - Pirates: 5 
Rockies: 9 - Cardinals: 12 

